In [1]:
import numpy as np
import pandas as pd

In [2]:
dados = pd.read_csv("../data/chamada_regular_sisu_2022_1.csv", sep = "|", encoding="latin1")

/tmp/ipykernel_8878/2431920978.py:1: DtypeWarning: Columns (17,22,23,24,25,26,27,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  dados = pd.read_csv("../data/chamada_regular_sisu_2022_1.csv", sep = "|", encoding="latin1")


In [3]:
# Observando o estado inicial dos dados
dados.head()

,ANO,EDICAO,ETAPA,DS_ETAPA,CODIGO_IES,NOME_IES,SIGLA_IES,UF_IES,CODIGO_CAMPUS,NOME_CAMPUS,...,NOTA_L_COM_PESO,NOTA_CH_COM_PESO,NOTA_CN_COM_PESO,NOTA_M_COM_PESO,NOTA_R_COM_PESO,NOTA_CANDIDATO,NOTA_CORTE,CLASSIFICACAO,APROVADO,MATRICULA
0,2022,1,4,CHAMADA REGULAR,593,CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA CELSO S...,CEFET/RJ,RJ,1663,CEFET-RJ - MARIA DA GRAÇA,...,"564,8","600,5","1233,8","2844,4",1740,"634,86","718,99",61.0,N,PENDENTE
1,2022,1,4,CHAMADA REGULAR,593,CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA CELSO S...,CEFET/RJ,RJ,1663,CEFET-RJ - MARIA DA GRAÇA,...,650,"669,9","1236,6","2794,8",1920,"661,03","718,99",45.0,N,PENDENTE
2,2022,1,4,CHAMADA REGULAR,593,CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA CELSO S...,CEFET/RJ,RJ,1663,CEFET-RJ - MARIA DA GRAÇA,...,"618,7","602,2","1332,6",2716,2640,"719,05","718,99",14.0,S,NÃO COMPARECEU
3,2022,1,4,CHAMADA REGULAR,593,CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA CELSO S...,CEFET/RJ,RJ,1663,CEFET-RJ - MARIA DA GRAÇA,...,"543,6","514,6",911,"2651,6",1560,"561,89","718,99",90.0,N,PENDENTE
4,2022,1,4,CHAMADA REGULAR,593,CENTRO FEDERAL DE EDUCAÇÃO TECNOLÓGICA CELSO S...,CEFET/RJ,RJ,1663,CEFET-RJ - MARIA DA GRAÇA,...,"637,8","741,4","1190,8","2686,8",1920,"652,44","718,99",51.0,N,PENDENTE


A partir dos dados iniciais do SISU, vamos fazer uma análise exploratória dos dados para responder as seguintes perguntas: 
- Qual a porcentagem de aprovados?
- Qual a maior e menor nota de aprovação?
- Dos aprovados, quantos porcento foram aceitos na primeira opção? E na segunda?
- Qual a probabilidade de ser aprovado na segunda opção dado que foi reprovado na primeira?
- Qual a porcentagem de cotistas aplicantes?
- Destes, qual a porcentagem de aprovados?
- Qual a porcentagem de cotistas aprovados na primeira opção? E na segunda?
- Qual a probabilidade de ser aprovado na primeira opção sendo um cotista? E na segunda?
- Há diferença na probabilidade de ser aprovado em um curso dado as modalidades de concorrência (cotas, ampla concorrência, etc)?
- Há alguma diferença na probabilidade de ser aprovado em um curso dado o seu turno?
- Há alguma diferença na probabilidade de ser aprovado em um curso dado o seu grau?
- No geral, qual a probabilidade de ser aprovado dada diferentes notas de corte?

**Brincar com as notas e os pesos para aprovação.**

# Qual a taxa de aprovação do ENEM?
Obs: A taxa é considerando primeira e segunda escolha.

In [4]:
# Calculando a porcentagem de aprovados
round(((dados['APROVADO'].value_counts()[1]/dados.shape[0])*100), 2)

/tmp/ipykernel_8878/886975928.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  round(((dados['APROVADO'].value_counts()[1]/dados.shape[0])*100), 2)


10.18

# Qual a maior e a menor nota de aprovação?

In [5]:
# Alterando a vírula por um ponto
dados['NOTA_CANDIDATO'] = dados['NOTA_CANDIDATO'].str.replace(',', '.')

# Alterando o tipo do dado para float
dados['NOTA_CANDIDATO'] = dados['NOTA_CANDIDATO'].astype(float)

In [6]:
# Buscando a maior nota média de um aprovado
dados.query("APROVADO == 'S'")["NOTA_CANDIDATO"].max()

953.14

In [7]:
# Buscando a menor nota média de um aprovado
dados.query("APROVADO == 'S'")["NOTA_CANDIDATO"].min()

208.79

In [8]:
# Verificando o terceiro quartil das notas dos aprovados
dados.query("APROVADO == 'S'")["NOTA_CANDIDATO"].quantile(0.75)

690.78

In [9]:
# Verificando o primeiro quartil das notas dos aprovados
dados.query("APROVADO == 'S'")["NOTA_CANDIDATO"].quantile(0.50)

634.66

# Dos aprovados, quantos porcento foram aceitos na primeira opção? E na segunda?

In [10]:
# Calculando a porcentagem de aceitos na primeira opção de curso
round(((dados.query("APROVADO == 'S' and OPCAO == 1").shape[0]/dados.query("APROVADO == 'S'").shape[0]) * 100), 2)

63.85

In [11]:
# Calculando a porcentagem de aceitos na segunda opção de curso
round(((dados.query("APROVADO == 'S' and OPCAO == 2").shape[0]/dados.query("APROVADO == 'S'").shape[0]) * 100), 2)

36.15

# Qual a probabilidade de ser aprovado na segunda opção dado que foi reprovado na primeira?

In [12]:
# Buscando todos os reprovados na primeira opção de curso
reprovados_1_opcao = dados.query("APROVADO == 'N' and OPCAO == 1")

# Buscando todos os aprovados na segunda opção de curso
aprovados_2_opcao = dados.query("APROVADO == 'S' and OPCAO == 2")


In [14]:
# Obtendo a lista de inscritos aprovados e seus respectivos CPFs
df_inscritos_aprovados = aprovados_2_opcao.groupby("CPF").INSCRITO.value_counts().to_frame().reset_index()
df_inscritos_aprovados = df_inscritos_aprovados.drop(columns = "count")

# Obtendo a lista de inscritos reprovados e seus respectivos CPFs
df_inscritos_reprovados = reprovados_1_opcao.groupby("CPF").INSCRITO.value_counts().to_frame().reset_index()
df_inscritos_reprovados = df_inscritos_reprovados.drop(columns = "count")

#qtd_reprovados_opcao_1_aprovados_opcao_2 = 

In [17]:
# Obtendo a quantidade de inscritos aprovados
qtd_reprovados_opcao_1_aprovados_opcao_2 = 0
for inscrito, cpf in zip(df_inscritos_aprovados.INSCRITO, df_inscritos_aprovados.CPF):
    qtd = df_inscritos_reprovados[(df_inscritos_reprovados['CPF'] == cpf) & (df_inscritos_reprovados['INSCRITO'] == inscrito)].shape[0]
    if qtd > 0:
        qtd_reprovados_opcao_1_aprovados_opcao_2 += 1
    else:
        continue

KeyboardInterrupt: 

In [9]:

# Calculando a porcentagem de reprovados na primeira opção de curso
prcnt_reprovados_1_opcao = round(((reprovados_1_opcao.shape[0]/dados.shape[0]) * 100), 2)

# Calculando a porcentagem de aprovados na segunda opção de curso
prcnt_aprovados_2_opcao = round(((aprovados_2_opcao.shape[0]/dados.shape[0]) * 100), 2)

# Calculando a porcentagem de reprovados na primeira opção de curso que foram aprovados na segunda opção de curso
prcnt_reprovados_1_opcao_aprovados_2_opcao = round(((qtd_reprovados_opcao_1_aprovados_opcao_2/dados.shape[0]) * 100), 2)